In [12]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve, differential_evolution, minimize
from scipy.linalg import eig
import multiprocessing as mp
from functools import partial
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Ниже полностью численное моделирование диф.уравнения 1 - с н.у. в начале второй моды 2 - с н.у. из нуля

In [42]:
def system(t, y, Omega, gammas):
    A_1, A_2, A_3, A_4, A_5, rho_1, rho_2 = y
    d1dt = 1j*ALPHA*rho_1*A_2 - gammas[0]*A_1/2 - 1j*Omega 
    d2dt = 1j*ALPHA*rho_2*A_3 - gammas[0]*A_2/2 + 1j*ALPHA*np.conj(rho_1)*A_1
    d3dt = 1j*ALPHA*rho_1*A_4 - gammas[0]*A_3/2 + 1j*ALPHA*np.conj(rho_2)*A_2
    d4dt = 1j*ALPHA*rho_2*A_5 - gammas[0]*A_4/2 + 1j*ALPHA*np.conj(rho_1)*A_3
    d5dt = 1j*ALPHA*np.conj(rho_2)*A_4 - gammas[0]*A_5 
    ddt = 1j*BETA*(A_1*np.conj(A_2) + A_3*np.conj(A_4)) - gammas[1]/2
    dsdt = 1j*BETA*(A_2*np.conj(A_3) + A_4*np.conj(A_5)) - gammas[1]/2
    return [d1dt, d2dt, d3dt, d4dt, d5dt, ddt, dsdt]

gammas = np.array([10, 1])
ALPHA = 1
BETA = 1
omega_first = gammas[0] / 2 * np.abs(-1j * np.sqrt( gammas[0] * gammas[1] / (4 * ALPHA * BETA) ) )
omega_second = np.sqrt(gammas[0] * (gammas[1] * gammas[0] + gammas[0] * gammas[1])**2 / (16 * ALPHA * BETA * gammas[1])) 
Omega_values = np.linspace(0, 60, 50)
t_span = (0, 400)
t_eval = np.linspace(0, 400, 1000)
amps = []


for Omega in Omega_values:
    params = Omega, gammas 
    if 8*ALPHA*BETA*Omega**2 *gammas[1] / (2*gammas[0]*gammas[1])**2 - gammas[0] / 2 > 0:
        print(Omega)
        A_1_in = -1j* Omega / gammas[0]
        A_2_in = np.sqrt(gammas[0]*gammas[1] / 4 / ALPHA / BETA)
        A_3_in = np.sqrt(8*ALPHA*BETA*Omega**2 *gammas[1] / (2*gammas[0]*gammas[1])**2 - gammas[0] / 2)
        rho_1_in = 2*BETA / gammas[1] * np.abs(A_2_in) * np.abs(A_3_in)
        rho_2_in = 2*BETA / gammas[1] * np.abs(A_2_in) * np.abs(A_1_in)

        initial_guess = [A_1_in, A_2_in, A_3_in, 0, 0, rho_1_in, rho_2_in]
        # initial_guess = np.zeros(7, dtype='complex128')
        sol = solve_ivp(system, t_span, initial_guess, args=params, 
                t_eval=t_eval, method='RK45')
        amp = (sol.y)
        amps.append(np.array(amp))
        
    else: 
        pass

15.918367346938776
17.142857142857142
18.367346938775512
19.591836734693878
20.816326530612244
22.040816326530614
23.26530612244898
24.48979591836735
25.714285714285715
26.93877551020408
28.16326530612245
29.387755102040817
30.612244897959183
31.836734693877553
33.06122448979592
34.285714285714285
35.51020408163265
36.734693877551024
37.95918367346939
39.183673469387756
40.40816326530612
41.63265306122449
42.85714285714286
44.08163265306123
45.30612244897959
46.53061224489796
47.755102040816325
48.9795918367347
50.204081632653065
51.42857142857143
52.6530612244898
53.87755102040816
55.10204081632653
56.3265306122449
57.55102040816327
58.775510204081634
60.0


In [43]:
gammas = np.array([10, 1])
ALPHA = 1
BETA = 1
omega_first = gammas[0] / 2 * np.abs(-1j * np.sqrt( gammas[0] * gammas[1] / (4 * ALPHA * BETA) ) )
omega_second = np.sqrt(gammas[0] * (gammas[1] * gammas[0] + gammas[0] * gammas[1])**2 / (16 * ALPHA * BETA * gammas[1])) 
Omega_values = np.linspace(0, 60, 50)
t_span = (0, 400)
t_eval = np.linspace(0, 400, 1000)
amps_stratch = []


for Omega in Omega_values:
    params = Omega, gammas 
    print(Omega)
    initial_guess = np.zeros(7, dtype='complex128')
    sol = solve_ivp(system, t_span, initial_guess, args=params, 
            t_eval=t_eval, method='RK45')
    amp = (sol.y)
    amps_stratch.append(np.array(amp))

0.0
1.2244897959183674
2.4489795918367347
3.673469387755102
4.8979591836734695
6.122448979591837
7.346938775510204
8.571428571428571
9.795918367346939
11.020408163265307
12.244897959183675
13.46938775510204
14.693877551020408
15.918367346938776
17.142857142857142
18.367346938775512
19.591836734693878
20.816326530612244
22.040816326530614
23.26530612244898
24.48979591836735
25.714285714285715
26.93877551020408
28.16326530612245
29.387755102040817
30.612244897959183
31.836734693877553
33.06122448979592
34.285714285714285
35.51020408163265
36.734693877551024
37.95918367346939
39.183673469387756
40.40816326530612
41.63265306122449
42.85714285714286
44.08163265306123
45.30612244897959
46.53061224489796
47.755102040816325
48.9795918367347
50.204081632653065
51.42857142857143
52.6530612244898
53.87755102040816
55.10204081632653
56.3265306122449
57.55102040816327
58.775510204081634
60.0


In [59]:
SET_IDX = 0  # индекс выбранного Omega
Omega_values = np.linspace(15.918367346938776, 60, len(amps))

# fig1 = go.Figure()
# if amps[SET_IDX] is not None:
#     for amp_idx in range(5):  # A1-A5
#         fig1.add_trace(
#             go.Scatter(
#                 x=t_eval,
#                 y=np.abs(amps[SET_IDX][amp_idx]),  # Берем модуль комплексного числа
#                 mode='lines',
#                 name=f'A{amp_idx+1}',
#                 line=dict(width=2)
#             )
#         )
    
#     fig1.update_layout(
#         title=f'Amplitude Dynamics for Ω={Omega_values[SET_IDX]:.2f}',
#         xaxis_title='Time',
#         yaxis_title='|Amplitude|',
#         template='plotly_white',
#         height=600,
#         width=800
#     )
#     fig1.show()
# else:
    # print(f"Нет данных для Ω={Omega_values[SET_IDX]:.2f}")

# 2. ГРАФИК: Финальные значения всех амплитуд
fig2 = go.Figure()
final_times = [t_eval[-1]] * len(Omega_values)  # Последний момент времени
omega_first = gammas[0] / 2 * np.abs(-1j * np.sqrt( gammas[0] * gammas[1] / (4 * ALPHA * BETA) ) )
omega_second = np.sqrt(gammas[0] * (gammas[1] * gammas[0] + gammas[0] * gammas[1])**2 / (16 * ALPHA * BETA * gammas[1])) 
fig2.add_vline(x= omega_first) 
fig2.add_vline(x= omega_second) 
for amp_idx in range(5):  # A1-A5
    final_values = []
    valid_omegas = []
    
    for i, amp_data in enumerate(amps):
        if amp_data is not None and len(amp_data) > amp_idx:
            final_values.append(np.abs(amp_data[amp_idx][-1]))  # Модуль последнего значения
            valid_omegas.append(Omega_values[i])
    
    if final_values:
        fig2.add_trace(
            go.Scatter(
                x=valid_omegas,
                y=final_values,
                mode='lines+markers',
                name=f'A{amp_idx+1} final amplitude',
                line=dict(width=2),
                marker=dict(size=8)
            )
        )

fig2.update_layout(
    title='Final Amplitude Values vs Ω',
    xaxis_title='Ω',
    yaxis_title='Final Amplitude |A|',
    template='plotly_white',
    height=600,
    width=800
)
fig2.show()

# # 3. ГРАФИК: Все амплитуды A1-A5 на отдельных subplots
# fig3 = make_subplots(
#     rows=3, cols=2,
#     subplot_titles=('A₁ amplitude', 'A₂ amplitude', 'A₃ amplitude', 
#                    'A₄ amplitude', 'A₅ amplitude'),
#     specs=[[{}, {}], [{}, {}], [{"colspan": 2}, None]]
# )

# for amp_idx in range(5):  # A1-A5
#     row = (amp_idx // 2) + 1
#     col = (amp_idx % 2) + 1
    
#     for i, amp_data in enumerate(amps):
#         if amp_data is not None and len(amp_data) > amp_idx:
#             fig3.add_trace(
#                 go.Scatter(
#                     x=t_eval,
#                     y=np.abs(amp_data[amp_idx]),
#                     mode='lines',
#                     name=f'Ω={Omega_values[i]:.2f}',
#                     line=dict(width=2),
#                     showlegend=(amp_idx == 0)  # Легенда только на первом графике
#                 ),
#                 row=row, col=col
#             )

# fig3.update_layout(
#     title='All Amplitude Dynamics',
#     height=900,
#     width=1000,
#     template='plotly_white',
#     showlegend=True
# )

# # Обновляем подписи осей
# for i in range(1, 6):
#     fig3.update_xaxes(title_text='Time', row=(i-1)//2 + 1, col=(i-1)%2 + 1)
#     fig3.update_yaxes(title_text='|A|', row=(i-1)//2 + 1, col=(i-1)%2 + 1)

# fig3.show()

# # 4. ОТДЕЛЬНЫЕ ГРАФИКИ для каждой амплитуды
# for amp_idx in range(5):  # A1-A5
    # fig_individual = go.Figure()
    
    # for i, amp_data in enumerate(amps):
    #     if amp_data is not None and len(amp_data) > amp_idx:
    #         fig_individual.add_trace(
    #             go.Scatter(
    #                 x=t_eval,
    #                 y=np.abs(amp_data[amp_idx]),
    #                 mode='lines',
    #                 name=f'Ω={Omega_values[i]:.2f}',
    #                 line=dict(width=2)
    #             )
    #         )
    
    # fig_individual.update_layout(
    #     title=f'A{amp_idx+1} Amplitude Dynamics',
    #     xaxis_title='Time',
    #     yaxis_title=f'|A{amp_idx+1}|',
    #     template='plotly_white',
    #     height=600,
    #     width=800
    # )
    # fig_individual.show()

In [60]:
fig2 = go.Figure()
Omega_values = np.linspace(0, 60, 50)
final_times = [t_eval[-1]] * len(Omega_values)  # Последний момент времени
omega_first = gammas[0] / 2 * np.abs(-1j * np.sqrt( gammas[0] * gammas[1] / (4 * ALPHA * BETA) ) )
omega_second = np.sqrt(gammas[0] * (gammas[1] * gammas[0] + gammas[0] * gammas[1])**2 / (16 * ALPHA * BETA * gammas[1])) 
fig2.add_vline(x= omega_first) 
fig2.add_vline(x= omega_second) 
for amp_idx in range(5):  # A1-A5
    final_values = []
    
    for i, amp_data in enumerate(amps_stratch):
        if amp_data is not None and len(amp_data) > amp_idx:
            final_values.append(np.abs(amp_data[amp_idx][-1])) 
    
    if final_values:
        fig2.add_trace(
            go.Scatter(
                x=Omega_values,
                y=final_values,
                mode='lines+markers',
                name=f'A{amp_idx+1} final amplitude',
                line=dict(width=2),
                marker=dict(size=8)
            )
        )

fig2.update_layout(
    title='Final Amplitude Values vs Ω',
    xaxis_title='Ω',
    yaxis_title='Final Amplitude |A|',
    template='plotly_white',
    height=600,
    width=800
)
fig2.show()

### Ниже склейка аналитического решения младших мод с полностью численным решением, либо с частично алгебраическим


In [45]:
def A1_st(gammas, Omega, order):
    if order == 'zero':
        return -2j * Omega / gammas[0]
    if order == 'first':
        return -1j * np.sqrt( gammas[0] * gammas[1] / (4 * ALPHA * BETA) ) 
    if order == 'second':
        return -2j * Omega * gammas[1] / (gammas[1] * gammas[0] + gammas[0] * gammas[1])

def A2_st(gammas, Omega, order):
    if order == 'zero':
        return 0
    if order == 'first':
        return np.sqrt((Omega - gammas[0] / 2 * np.abs(A1_st(gammas, Omega, order))) / ALPHA / BETA / 2 * gammas[1] / np.abs(A1_st(gammas, Omega, order)))  # phi_2 = 0
        
    if order == 'second':
        return 1j * np.sqrt(gammas[1] * gammas[0] / (4 * ALPHA * BETA))  # phi_2 = np.pi / 2

def A3_st(gammas, Omega, order):
    if order == 'zero' or order == 'first':
        return 0
    if order == 'second':
        return np.sqrt(2 * ALPHA * BETA / gammas[1] * np.abs(A1_st(gammas, Omega, order))**2 - gammas[0] / 2)  # phi_3 = 0, phi_rho^2 = phi_rho^1 = - np.pi / 2
    
def rho1_st(gammas, Omega, order):
    if order == 'zero':
        return 0
    if order == 'first':
        return np.abs(A1_st(gammas, Omega, order)) * np.abs(A2_st(gammas, Omega, order)) * 2 * BETA / gammas[3]  # phi_rho^1 = 0
    if order == 'second':
        return -2j * BETA / gammas[3] * np.abs(A2_st(gammas, Omega, order)) * np.abs(A1_st(gammas, Omega, order)) 

def rho2_st(gammas, Omega, order):
    if order == 'zero' or order == 'first':
        return 0
    else:
        return -2j * BETA / gammas[4] * np.abs(A2_st(gammas, Omega, order)) * np.abs(A3_st(gammas, Omega, order))

def build_matrix(Omega, gammas, order):
    M = np.zeros((10, 10), dtype=complex)
    
    M[0, 0] = -1 / 2 * gammas[0]
    M[0, 1] = 1j * ALPHA * rho1_st(gammas, Omega, order)
    M[0, 3] = 1j * ALPHA * A2_st(gammas, Omega, order)

    M[1, 1] = -1 / 2 * gammas[1]
    M[1, 0] = 1j * ALPHA * np.conj(rho1_st(gammas, Omega, order))
    M[1, 2] = 1j * ALPHA * rho2_st(gammas, Omega, order)
    M[1, 4] = 1j * ALPHA * A3_st(gammas, Omega, order)
    M[1, 8] = 1j * ALPHA * A1_st(gammas, Omega, order)

    M[2, 2] = -1 / 2 * gammas[2]
    M[2, 1] = 1j * ALPHA * np.conj(rho2_st(gammas, Omega, order))
    M[2, 9] = 1j * ALPHA * A2_st(gammas, Omega, order)

    M[3, 3] = -1 / 2 * gammas[3]
    M[3, 0] = 1j * BETA * A2_st(gammas, Omega, order)
    M[3, 6] = 1j * BETA * A1_st(gammas, Omega, order)

    M[4, 4] = -1 / 2 * gammas[4]
    M[4, 1] = 1j * BETA * np.conj(A3_st(gammas, Omega, order))
    M[4, 7] = 1j * BETA * A2_st(gammas, Omega, order)

    M[5, 5] = -1 / 2 * gammas[0]          
    M[5, 6] = -1j * ALPHA * np.conj(rho1_st(gammas, Omega, order))  
    M[5, 8] = -1j * ALPHA * np.conj(A2_st(gammas, Omega, order))         

    M[6, 6] = -1 / 2 * gammas[1]
    M[6, 3] = -1j * ALPHA * np.conj(A1_st(gammas, Omega, order))         
    M[6, 5] = -1j * ALPHA * rho1_st(gammas, Omega, order)     
    M[6, 7] = -1j * ALPHA * np.conj(rho2_st(gammas, Omega, order)) 
    M[6, 9] = -1j * ALPHA * np.conj(A3_st(gammas, Omega, order)) 

    M[7, 7] = -1 / 2 * gammas[2]  
    M[7, 4] = -1j * ALPHA * np.conj(A2_st(gammas, Omega, order))           
    M[7, 6] = -1j * ALPHA * rho2_st(gammas, Omega, order)        

    M[8, 8] = -1 / 2 * gammas[3]        
    M[8, 1] = -1j * BETA * np.conj(A1_st(gammas, Omega, order))               
    M[8, 5] = -1j * BETA * np.conj(A2_st(gammas, Omega, order))     

    M[9, 9] = -1 / 2 * gammas[4]    
    M[9, 2] = -1j * BETA * np.conj(A2_st(gammas, Omega, order))          
    M[9, 6] = -1j * BETA * A3_st(gammas, Omega, order)
    # for row in M:
    #     print(row)
    return M

def stability_analysis(Omega_range, gamma_range, gamma_index, order):
    stability_map = np.zeros((len(Omega_range), len(gamma_range)))
    
    for i, Omega in enumerate(Omega_range):
        for j, gamma_val in enumerate(gamma_range):
            current_gammas = np.ones(5)

            current_gammas[gamma_index] = gamma_val
            
            M = build_matrix(Omega, current_gammas, order)
            # print(M)
            try:
                eigenvalues = eig(M)[0]
                max_real = np.max(np.real(eigenvalues))
                print(np.real(eigenvalues))
                stability_map[i, j] = 1 if max_real < TOLERANCY else 0
                print(1 if max_real < 0 else 0)
                # if stability_map[i, j] == 1:
                #     print(max_real)
            except:
                stability_map[i, j] = np.nan

            
    return stability_map

def analytical_curve1(gamma_val, gamma_index): # second mode existance 
    g = np.ones(5)
    g[gamma_index] = gamma_val
    return np.sqrt(1/(16 * ALPHA * BETA) * g[1] * (g[4] * g[2] + g[0] * g[3])**2 / g[3])

def analytical_curve2(gamma_val, gamma_index): # zero mode stability
    g = np.ones(5)
    g[gamma_index] = gamma_val
    numerator = g[0]**2 * (1 + g[1]/g[2] + g[3]/g[2]) * (g[3]*g[2] + g[1]*g[3] + g[1]*g[2])
    return np.sqrt(1/(16 * ALPHA * BETA) * numerator)

In [57]:
omega_first = gammas[0] / 2 * np.abs(-1j * np.sqrt(gammas[0] * gammas[1] / (4 * ALPHA * BETA)))
omega_second = np.sqrt(gammas[0] * (gammas[1] * gammas[0] + gammas[0] * gammas[1])**2 / (16 * ALPHA * BETA * gammas[1]))

# Создаем один график
fig = go.Figure()

# Первая часть: аналитические решения (0 до 15.918...)
Omega_values_analytical = np.linspace(0, 15.918367346938776, 50)

# Диапазон 1: до omega_first
mask1 = (Omega_values_analytical <= omega_first)
A1_zero = np.array([np.abs(A1_st(gammas, Omega, 'zero')) for Omega in Omega_values_analytical[mask1]])
A2_zero = np.array([np.abs(A2_st(gammas, Omega, 'zero')) for Omega in Omega_values_analytical[mask1]])
A3_zero = np.array([np.abs(A3_st(gammas, Omega, 'zero')) for Omega in Omega_values_analytical[mask1]])

fig.add_trace(go.Scatter(x=Omega_values_analytical[mask1], y=A1_zero, mode='lines', 
                         name='|A1| analytical', line=dict(color='red')))
fig.add_trace(go.Scatter(x=Omega_values_analytical[mask1], y=A2_zero, mode='lines', 
                         name='|A2| analytical', line=dict(color='green')))
fig.add_trace(go.Scatter(x=Omega_values_analytical[mask1], y=A3_zero, mode='lines', 
                         name='|A3| analytical', line=dict(color='blue')))

# Диапазон 2: между omega_first и omega_second
mask2 = (Omega_values_analytical >= omega_first) & (Omega_values_analytical <= omega_second)
A1_first = np.array([np.abs(A1_st(gammas, Omega, 'first')) for Omega in Omega_values_analytical[mask2]])
A2_first = np.array([np.abs(A2_st(gammas, Omega, 'first')) for Omega in Omega_values_analytical[mask2]])
A3_first = np.array([np.abs(A3_st(gammas, Omega, 'first')) for Omega in Omega_values_analytical[mask2]])

fig.add_trace(go.Scatter(x=Omega_values_analytical[mask2], y=A1_first, mode='lines', 
                         name='|A1| analytical', line=dict(color='red'), showlegend=False))
fig.add_trace(go.Scatter(x=Omega_values_analytical[mask2], y=A2_first, mode='lines', 
                         name='|A2| analytical', line=dict(color='green'), showlegend=False))
fig.add_trace(go.Scatter(x=Omega_values_analytical[mask2], y=A3_first, mode='lines', 
                         name='|A3| analytical', line=dict(color='blue'), showlegend=False))

# Диапазон 3: после omega_second
mask3 = Omega_values_analytical >= omega_second
A1_second = np.array([np.abs(A1_st(gammas, Omega, 'second')) for Omega in Omega_values_analytical[mask3]])
A2_second = np.array([np.abs(A2_st(gammas, Omega, 'second')) for Omega in Omega_values_analytical[mask3]])
A3_second = np.array([np.abs(A3_st(gammas, Omega, 'second')) for Omega in Omega_values_analytical[mask3]])

fig.add_trace(go.Scatter(x=Omega_values_analytical[mask3], y=A1_second, mode='lines', 
                         name='|A1| analytical', line=dict(color='red'), showlegend=False))
fig.add_trace(go.Scatter(x=Omega_values_analytical[mask3], y=A2_second, mode='lines', 
                         name='|A2| analytical', line=dict(color='green'), showlegend=False))
fig.add_trace(go.Scatter(x=Omega_values_analytical[mask3], y=A3_second, mode='lines', 
                         name='|A3| analytical', line=dict(color='blue'), showlegend=False))

# Вторая часть: численные решения (15.918... до 60)
Omega_values_numerical = np.linspace(0, 60, 50)

# Добавляем численные данные для A1-A5
colors = ['red', 'green', 'blue', 'orange', 'purple']
for amp_idx in range(5):
    final_values = []
    valid_omegas = []
    
    for i, amp_data in enumerate(amps_stratch):
        if amp_data is not None and len(amp_data) > amp_idx:
            final_values.append(np.abs(amp_data[amp_idx][-1]))
            valid_omegas.append(Omega_values_numerical[i])
    
    if final_values:
        fig.add_trace(go.Scatter(
            x=valid_omegas,
            y=final_values,
            mode='lines+markers',
            name=f'A{amp_idx+1} numerical',
            line=dict(width=2, color=colors[amp_idx % len(colors)], dash='dash'),
            marker=dict(size=8, color=colors[amp_idx % len(colors)])
        ))

# Вертикальные линии
fig.add_vline(x=omega_first, line=dict(dash='dot', color='black'))
fig.add_vline(x=omega_second, line=dict(dash='dash', color='black'))

# Настройки графика
fig.update_layout(
    title='Amplification of modes: Analytical and Numerical Solutions',
    xaxis_title='Ω (Накачка)',
    yaxis_title='Амплитуда',
    template='plotly_white',
    height=600,
    width=1000
)

fig.show()

### Ниже склейка частично алгебраического и польностью численного подхода

In [74]:
def kappa(gammas):
    return np.sqrt(gammas[0] * gammas[1] / 4 / ALPHA / BETA)

def theta(gammas, Omega):
    return (Omega * gammas[1] / ALPHA / BETA / 2) ** (1/3)

def A_2_A_4_sys(vars, gammas, Omega):
    x, y = vars
    k = kappa(gammas)
    t = theta(gammas, Omega)
    
    c_6 = - k**4*x - k**2*y**2 + k**6 - x*y*k**2
    b_4 = y**2 + k**2 * x - k**4
    a_2 = k**2 - y
    
    term = k**2 * c_6**2 / t**6
    
    eq1 = b_4**2 + b_4*a_2*y - a_2**2*x*y - a_2*y**2*x - term
    eq2 = a_2*b_4*x + a_2**2*x*y - a_2*x**2*y - x**2*y**2 - term
    
    return [eq1, eq2]

def A_2_A_4(gammas, Omega, method='de', tol=1e-20, maxiter=500):
    try:
        denominator = 16 * ALPHA * BETA * gammas[1]
        if denominator == 0:
            return (np.nan, np.nan)

            
        lower_bound = max(0, Omega) 
        upper_bound = max(Omega, lower_bound + 1) 
        bounds = [(lower_bound, upper_bound), (lower_bound, upper_bound)]
        
        def objective(vars):
            eq1, eq2 = A_2_A_4_sys(vars, gammas, Omega)
            return eq1**2 + eq2**2
        
        if method == 'de':
            result = differential_evolution(objective, bounds, tol=tol, maxiter=maxiter)
            solution = result.x
            
        elif method == 'hybrid':
            result_de = differential_evolution(objective, bounds, tol=tol, maxiter=maxiter//2)
            result_fsolve = fsolve(lambda vars: A_2_A_4_sys(vars, gammas, Omega), 
                                 result_de.x, 
                                 xtol=tol, 
                                 maxfev=maxiter//2)
            solution = result_fsolve
            
        x, y = solution

        print(x,y)
        return (np.sqrt(x), np.sqrt(y))
        
    except Exception as e:
        print(f"Ошибка при вычислении A_2_A_4: {str(e)}")
        return (np.nan, np.nan)

def calc_3rd_order_st_amps(gammas, Omega):
    A_2, A_4 = A_2_A_4(gammas, Omega)
    k = kappa(gammas)
    t = theta(gammas, Omega)
    c_6 = - k**4*A_2**2 - k**2*A_4**4 + k**6 - A_2**2*A_4**2*k**2
    b_4 = A_4**4 + k**2 * A_2**2 - k**4
    a_2 = k**2 - A_4**2

    A2_st = A_2                                               # phi_2 = 0
    A4_st = A_4                                               # phi_4 = 0
    A3_st = 1j*t**3 * A4_st * np.conj(A2_st) * a_2 / c_6      # phi_3 = pi/2
    A1_st = 1j*t**3 * b_4 / c_6                               # phi_1 = pi/2
    A5_st = 1j*t**3 * A4_st * A4_st * A2_st * A2_st / c_6     # phi_5 = pi/2
    rho1_st = 2j / gammas[1] * BETA * (A1_st * np.conj(A2_st) + A3_st * np.conj(A4_st))
    rho2_st = 2j / gammas[1] * BETA * (A2_st * np.conj(A3_st) + A4_st * np.conj(A5_st))

    return (A1_st, A2_st, A3_st, A4_st, A5_st, rho1_st, rho2_st)

def build_matrix(gammas, Omega):
    A1_st, A2_st, A3_st, A4_st, A5_st, rho1_st, rho2_st = calc_3rd_order_st_amps(gammas, Omega)
    M = np.zeros((14, 14), dtype=complex)
    
    M[0,0] = -gammas[0] / 2
    M[0,1] = 1j*ALPHA*rho1_st
    M[0,5] = 1j*ALPHA*A2_st

    M[1,0] = 1j*ALPHA*np.conj(rho1_st)
    M[1,1] = -gammas[0] / 2
    M[1,2] = 1j*ALPHA*rho2_st
    M[1,6] = 1j*ALPHA*A3_st
    M[1,12] = 1j*ALPHA*A1_st

    M[2,1] = 1j*ALPHA*np.conj(rho2_st)
    M[2,2] = -gammas[0] / 2
    M[2,3] = 1j*ALPHA*rho1_st
    M[2,5] = 1j*ALPHA*A4_st
    M[2,13] = 1j*ALPHA*A2_st

    M[3,2] = 1j*ALPHA*np.conj(rho1_st)
    M[3,3] = -gammas[0] / 2
    M[3,4] = 1j*ALPHA*rho2_st
    M[3,6] = 1j*ALPHA*A5_st
    M[3,12] = 1j*ALPHA*A3_st

    M[4,3] = 1j*ALPHA*np.conj(rho2_st)
    M[4,4] = -gammas[0] / 2
    M[4,13] = 1j*ALPHA*A4_st

    M[5,0] = 1j*BETA*np.conj(A2_st)
    M[5,2] = 1j*BETA*np.conj(A4_st)
    M[5,5] = -gammas[1] / 2
    M[5,8] = 1j*BETA*A1_st
    M[5,10] = 1j*BETA*A3_st

    M[6,1] = 1j*BETA*np.conj(A3_st)
    M[6,3] = 1j*BETA*np.conj(A5_st)
    M[6,6] = -gammas[1] / 2
    M[6,9] = 1j*BETA*A2_st
    M[6,11] = 1j*BETA*A4_st

    M[7,7] = -np.conj(gammas[0]) / 2
    M[7,8] = -1j*ALPHA*np.conj(rho1_st)
    M[7,12] = -1j*ALPHA*np.conj(A2_st)

    M[8,7] = -1j*ALPHA*rho1_st
    M[8,8] = -np.conj(gammas[0]) / 2
    M[8,9] = -1j*ALPHA*np.conj(rho2_st)
    M[8,13] = -1j*ALPHA*np.conj(A3_st)
    M[8,5] = -1j*ALPHA*np.conj(A1_st)

    M[9,8] = -1j*ALPHA*rho2_st
    M[9,9] = -np.conj(gammas[0]) / 2
    M[9,10] = -1j*ALPHA*np.conj(rho1_st)
    M[9,12] = -1j*ALPHA*np.conj(A4_st)
    M[9,6] = -1j*ALPHA*np.conj(A2_st)

    M[10,9] = -1j*ALPHA*rho1_st
    M[10,10] = -np.conj(gammas[0]) / 2
    M[10,11] = -1j*ALPHA*np.conj(rho2_st)
    M[10,13] = -1j*ALPHA*np.conj(A5_st)
    M[10,5] = -1j*ALPHA*np.conj(A3_st)

    M[11,10] = -1j*ALPHA*rho2_st
    M[11,11] = -np.conj(gammas[0]) / 2
    M[11,6] = -1j*ALPHA*np.conj(A4_st)

    M[12,7] = -1j*BETA*A2_st
    M[12,9] = -1j*BETA*A4_st
    M[12,12] = -np.conj(gammas[1]) / 2
    M[12,1] = -1j*BETA*np.conj(A1_st)
    M[12,3] = -1j*BETA*np.conj(A3_st)

    M[13,8] = -1j*BETA*A3_st
    M[13,10] = -1j*BETA*A5_st
    M[13,13] = -np.conj(gammas[1]) / 2
    M[13,2] = -1j*BETA*np.conj(A2_st)
    M[13,4] = -1j*BETA*np.conj(A4_st)

    return M, (A1_st, A2_st, A3_st, A4_st, A5_st, rho1_st, rho2_st)

In [78]:
fig = go.Figure()
Omega_values_am = np.linspace(0, 15.918367346938776, 500)

# am = []
# for Omega in Omega_values_am:
#     M, amp = build_matrix(gammas, Omega)
#     am.append(amp)
#     print(amp)

# am = np.array(am)
colors = ['red', 'green', 'blue', 'orange', 'purple']
for i in range(5):
    fig.add_trace(go.Scatter(
        x=Omega_values_am,
        y=np.abs(am[:, i]),
        name=f'A{i+1} algebraic',
        mode='lines',
        line=dict(width=2, dash='dash', color=colors[i])
    ))

Omega_values_amps = np.linspace(15.918367346938776, 60, 50)

for i in range(5):
    final_values = []
    valid_omegas = []
    
    for j, amp_data in enumerate(amps):
        if amp_data is not None and len(amp_data) > i:
            final_values.append(np.abs(amp_data[i][-1]))
            valid_omegas.append(Omega_values_amps[j])
    
    if final_values:
        fig.add_trace(go.Scatter(
            x=valid_omegas,
            y=final_values,
            mode='lines+markers',
            name=f'A{i+1} full numerical',
            line=dict(width=2, dash='dot', color=colors[i]),
            marker=dict(size=6)
        ))

fig.add_vline(x=omega_first, line=dict(dash='dot', color='black'))
fig.add_vline(x=omega_second, line=dict(dash='dash', color='black'))

fig.update_layout(
    title='Зависимость амплитуд от накачки Ω: Все решения',
    xaxis_title='Накачка Ω',
    yaxis_title='Амплитуда',
    legend_title='Типы решений',
    hovermode='x unified', 
    template='plotly_white',
    height=600,
    width=1000
)

fig.update_xaxes(showspikes=True, spikecolor='gray', spikesnap='cursor', spikemode='across')
fig.update_yaxes(showspikes=True, spikecolor='gray', spikesnap='cursor', spikemode='across')

fig.show()